In [1]:
import numpy as np
import skfuzzy as fuzz
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Define image dimensions and batch size
img_height, img_width = 128, 128
batch_size = 32
# Define directories for training and testing data
train_data_dir = "../dataset/train"
test_data_dir = "../dataset/test"

In [3]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255  # Normalize pixel values
)

# Data augmentation for testing images (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255)


In [4]:
# Create data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True  # No need to shuffle for evaluation
)



Found 1174 images belonging to 14 classes.
Found 428 images belonging to 14 classes.


# TRAINING PHASE

In [5]:
# Define the VGG16 base model
from tensorflow.keras.applications import EfficientNetV2L

def create_EfficientNetV2L_model():
    EfficientNetV2L_model = EfficientNetV2L(
        weights='imagenet', 
        include_top=False, 
        input_shape=(img_height, img_width, 3)
    )
    EfficientNetV2L_model.trainable = False


    model = Sequential([
        EfficientNetV2L_model,
        GlobalAveragePooling2D(),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(14, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print(model.summary())
    return model




In [6]:
#MODEL fit
model = create_EfficientNetV2L_model()
model.fit(train_generator, epochs=20, validation_data=test_generator)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-l (Functiona  (None, 4, 4, 1280)       117746848 
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 64)                81984     
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                        

In [11]:
# predicting train dataset 
test_pridiction = model.predict(test_generator)

# true labels 
test_true_labels = test_generator.classes




14/14 [==============================] - 4s 283ms/step


In [12]:
print( test_pridiction.shape)
print( test_true_labels.shape)

#calculating accuracy
accuracy = np.mean(np.argmax(test_pridiction, axis=1) == test_true_labels)
print(accuracy)


(428, 14)
(428,)
0.09813084112149532
